In [1]:
##Classification Exercise using PyTorch
## Reference: CoderzColumn Tutorial

## Steps involved:
## Load Dataset(Breast Cancer Dataset from scikit-learn)
## Normalize Data - Subtracting the mean and dividing by Standard Deviation
## Initialize weights - use requires_grad and torch.rand
## Activation of Hidden Layers - Activation function using ReLu
## Activation of Last Layer - using Sigmoid function
## Define Single Layer of NN
## Define Single Forward Pass through Data to Make Predictions
## Define Loss functioni (log_loss(actual,predictions) function is being used)
## Train Neural Network - Important aspects - epochs, learning rate, layer sizes
## Make Predictions
## Evaluate Performance of NN
## Define Batch size and Train data in Batches
## Make Predictions in Batches
## Evaluate Performance (Calculate Accuracy)


import torch #root package
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader #dataset representation and loading

print("PyTorch Version : {}".format(torch.__version__))


PyTorch Version : 2.3.0+cu118


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device : {}".format(device))

Device : cpu


In [3]:
##Load Dataset, Breast Cancer Dataset

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

X, Y = datasets.load_breast_cancer(return_X_y=True)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, stratify=Y, random_state=123)

X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32),\
                                   torch.tensor(X_test, dtype=torch.float32),\
                                   torch.tensor(Y_train, dtype=torch.float32),\
                                   torch.tensor(Y_test, dtype=torch.float32)

samples, features = X_train.shape
classes = Y_test.unique()

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


(torch.Size([455, 30]),
 torch.Size([114, 30]),
 torch.Size([455]),
 torch.Size([114]))

In [5]:
##Print Samples, features, classes

samples, features, classes

(455, 30, tensor([0., 1.]))

In [6]:
##Normalize Data

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean)/ std
X_test = (X_test - mean)/ std

In [7]:
##Initialize Model Weights

def InitializeWeights(layer_sizes, scale=0.1):
    weights = []
    for i, units in enumerate(layer_sizes):
        if i==0:
            w = torch.rand(units,features, dtype=torch.float32)
        else:
            w = torch.rand(units,layer_sizes[i-1], dtype=torch.float32)

        b = torch.rand(units, dtype=torch.float32)

        if scale: ## Scale weights
            w = w*scale
            b = b*scale

        w.requires_grad=True ## Set requires grad after weights are updated with scale
        b.requires_grad=True

        weights.append([w,b])

    return weights

In [8]:
##Activation for Hidden Layers

def Relu(tensor):
    return torch.maximum(tensor, torch.zeros_like(tensor)) # max(0,x)

In [9]:
## Activation for Last layers

def Sigmoid(tensor):
    return 1 / (1 + torch.exp(-tensor))
tensor = torch.tensor([1,2,3,4,5])

Sigmoid(tensor), torch.nn.Sigmoid()(tensor)

(tensor([0.7311, 0.8808, 0.9526, 0.9820, 0.9933]),
 tensor([0.7311, 0.8808, 0.9526, 0.9820, 0.9933]))

In [10]:
##Single Layer of NN

def LinearLayer(weights, input_data, activation=lambda x: x):
    w, b = weights
    out = torch.matmul(input_data, w.T) + b ## Multiply input by weights and add bias to it.
    return activation(out) ## Apply activation at last

In [11]:
## Single Forward Pass through Data to Make Prediction

def ForwardPass(weights, input_data):
    layer_out = input_data

    for i in range(len(weights[:-1])):
        layer_out = LinearLayer(weights[i], layer_out, Relu) ## Hidden Layer

    preds = LinearLayer(weights[-1], layer_out, Sigmoid) ## Final Layer

    return torch.clamp(preds.squeeze(), 0.01, 0.99)



In [12]:
## Define Loss function

def NegLogLoss(actual, preds):
    loss = - actual * torch.log(preds) - (1 - actual) * torch.log(1 - preds)
    return loss.mean()

y1 = torch.tensor([1,1,0, 0,1])
y2 = torch.tensor([0.7,0.1,0.69, 0.1,0.23])

NegLogLoss(y1, y2)

tensor(1.0811)

In [13]:
from sklearn.metrics import log_loss

log_loss(y1.detach().numpy(), y2.detach().numpy())

1.0810959013881432

In [14]:
from torch import autograd

def TrainModel(X, Y, learning_rate, epochs):

    for i in range(1, epochs+1):
        preds = ForwardPass(weights, X) ## Make Predictions by forward pass through network

        loss = NegLogLoss(Y, preds) ## Calculate Loss

        loss.backward() ## Calculate Gradients

        with torch.no_grad():
            for j in range(len(weights)): ## Update Weights
                weights[j][0] -= learning_rate * weights[j][0].grad ## Update Weights
                weights[j][1] -= learning_rate * weights[j][1].grad ## Update Biases

                weights[j][0].grad = None
                weights[j][1].grad = None

        if i % 100 == 0: ## Print NegLogLoss every 100 epochs
            print("NegLogLoss : {:.2f}".format(loss))

In [15]:
torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 2500
learning_rate = torch.tensor(1/1e2) # 0.01
layer_sizes = [5,10,15,1] ## Layer sizes including last layer

weights = InitializeWeights(layer_sizes) ## Initialize Weights

TrainModel(X_train, Y_train, learning_rate, epochs)

NegLogLoss : 0.69
NegLogLoss : 0.68
NegLogLoss : 0.67
NegLogLoss : 0.67
NegLogLoss : 0.66
NegLogLoss : 0.66
NegLogLoss : 0.65
NegLogLoss : 0.64
NegLogLoss : 0.62
NegLogLoss : 0.58
NegLogLoss : 0.50
NegLogLoss : 0.42
NegLogLoss : 0.35
NegLogLoss : 0.31
NegLogLoss : 0.27
NegLogLoss : 0.24
NegLogLoss : 0.22
NegLogLoss : 0.20
NegLogLoss : 0.18
NegLogLoss : 0.17
NegLogLoss : 0.16
NegLogLoss : 0.15
NegLogLoss : 0.14
NegLogLoss : 0.13
NegLogLoss : 0.13


In [16]:
##Make Predictions

train_preds = ForwardPass(weights, X_train)

train_preds = torch.as_tensor(train_preds > 0.5, dtype=torch.float32)

train_preds[:5], Y_train[:5]

(tensor([1., 1., 0., 1., 1.]), tensor([1., 1., 0., 0., 1.]))

In [17]:
test_preds = ForwardPass(weights, X_test)

test_preds = torch.as_tensor(test_preds > 0.5, dtype=torch.float32)

test_preds[:5], Y_test[:5]

(tensor([0., 0., 1., 1., 1.]), tensor([0., 0., 1., 1., 1.]))

In [18]:
##Evaluate Performance of Neural Network

from sklearn.metrics import accuracy_score

print("Train Accuracy : {:.2f}".format(accuracy_score(Y_train, train_preds)))
print("Test  Accuracy : {:.2f}".format(accuracy_score(Y_test, test_preds)))

Train Accuracy : 0.98
Test  Accuracy : 0.98


In [19]:
##Train data in batches

def UpdateWeights(weights, learning_rate):
    with torch.no_grad():
        for j in range(len(weights)): ## Update Weights
            weights[j][0] -= learning_rate * weights[j][0].grad ## Update Weights
            weights[j][1] -= learning_rate * weights[j][1].grad ## Update Biases

            weights[j][0].grad = None
            weights[j][1].grad = None

def TrainModelInBatches(X, Y, learning_rate, epochs, batch_size=32):
    for i in range(1, epochs+1):
        batches = torch.arange((X.shape[0]//batch_size)+1) ### Batch Indices

        losses = [] ## Record loss of each batch
        for batch in batches:
            if batch != batches[-1]:
                start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
            else:
                start, end = int(batch*batch_size), None

            X_batch, Y_batch = X[start:end], Y[start:end] ## Single batch of data

            preds = ForwardPass(weights, X_batch) ## Make Predictions by forward pass through network

            loss = NegLogLoss(Y_batch, preds) ## Calculate Loss
            losses.append(loss) ## Record Loss
            loss.backward() ## Calculate Gradients

            UpdateWeights(weights, learning_rate) ## Update Weights

        if i % 100 == 0: ## Print NegLogLoss every 100 epochs
            print("NegLogLoss : {:.2f}".format(torch.tensor(losses).mean()))



torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 2500
learning_rate = torch.tensor(1/1e3) # 0.01
layer_sizes = [5,10,15, 1] ## Layer sizes including last layer
weights = InitializeWeights(layer_sizes) ## Initialize Weights

TrainModelInBatches(X_train, Y_train, learning_rate, epochs)
            

NegLogLoss : 0.68
NegLogLoss : 0.67
NegLogLoss : 0.66
NegLogLoss : 0.65
NegLogLoss : 0.65
NegLogLoss : 0.63
NegLogLoss : 0.58
NegLogLoss : 0.46
NegLogLoss : 0.35
NegLogLoss : 0.28
NegLogLoss : 0.24
NegLogLoss : 0.20
NegLogLoss : 0.18
NegLogLoss : 0.16
NegLogLoss : 0.15
NegLogLoss : 0.13
NegLogLoss : 0.12
NegLogLoss : 0.12
NegLogLoss : 0.11
NegLogLoss : 0.10
NegLogLoss : 0.10
NegLogLoss : 0.09
NegLogLoss : 0.09
NegLogLoss : 0.09
NegLogLoss : 0.08


In [20]:
##Make Predictions in Batches

def MakePredictions(input_data, batch_size=32):
    batches = torch.arange((input_data.shape[0]//batch_size)+1) ### Batch Indices

    with torch.no_grad(): ## Disables automatic gradients calculations
        preds = []
        for batch in batches:
            if batch != batches[-1]:
                start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
            else:
                start, end = int(batch*batch_size), None

            X_batch = input_data[start:end]

            preds.append(ForwardPass(weights, X_batch))

    return preds
    

In [21]:
test_preds = MakePredictions(X_test) ## Make Predictions on test dataset

test_preds = torch.cat(test_preds) ## Combine all batch predictions

test_preds = torch.as_tensor(test_preds > 0.5, dtype=torch.float32) ## Convert Probabilities to class type

train_preds = MakePredictions(X_train) ## Make Predictions on train dataset

train_preds = torch.cat(train_preds) ## Combine all batch predictions

train_preds = torch.as_tensor(train_preds > 0.5, dtype=torch.float32) ## Convert Probabilities to class type

In [22]:
##Evaluate Performance

from sklearn.metrics import accuracy_score

print("Train Accuracy : {:.2f}".format(accuracy_score(Y_train, train_preds)))
print("Test  Accuracy : {:.2f}".format(accuracy_score(Y_test, test_preds)))

Train Accuracy : 0.98
Test  Accuracy : 0.98
